In [1]:
import sqlalchemy as db

In [2]:
engine = db.create_engine('sqlite:///mydb6.db')

In [3]:
conn = engine.connect()

In [4]:
metadata = db.MetaData()

In [5]:
cars = db.Table(
    'Car', metadata,
    db.Column('car_id', db.Integer, primary_key=True),
    db.Column('car_name', db.Text,),
    db.Column('car_country', db.Text),
    db.Column('car_mileage', db.Integer),
    db.Column('car_price', db.Integer)
)
cars

Table('Car', MetaData(), Column('car_id', Integer(), table=<Car>, primary_key=True, nullable=False), Column('car_name', Text(), table=<Car>), Column('car_country', Text(), table=<Car>), Column('car_mileage', Integer(), table=<Car>), Column('car_price', Integer(), table=<Car>), schema=None)

In [6]:
countrys = db.Table(
    'Country', metadata,
    db.Column('country_id', db.Integer, primary_key=True),
    db.Column('country_name', db.Text,),
    db.Column('country_capital', db.Text),
)
countrys

Table('Country', MetaData(), Column('country_id', Integer(), table=<Country>, primary_key=True, nullable=False), Column('country_name', Text(), table=<Country>), Column('country_capital', Text(), table=<Country>), schema=None)

In [7]:
metadata.create_all(engine)

In [8]:
insertion = cars.insert().values([
    {'car_name': 'Audi', 'car_country': 'Germany', 'car_mileage': 10000, 'car_price': 35500},
    {'car_name': 'BMW', 'car_country': 'Germany', 'car_mileage': 15600, 'car_price': 24500},
    {'car_name': 'Mercedes', 'car_country': 'Germany', 'car_mileage': 4000, 'car_price': 55500}

])
print(insertion)

INSERT INTO "Car" (car_name, car_country, car_mileage, car_price) VALUES (:car_name_m0, :car_country_m0, :car_mileage_m0, :car_price_m0), (:car_name_m1, :car_country_m1, :car_mileage_m1, :car_price_m1), (:car_name_m2, :car_country_m2, :car_mileage_m2, :car_price_m2)


In [9]:
conn.execute(insertion)

In [10]:
conn.execute(db.select(cars)).fetchall()

[(1, 'Audi', 'Germany', 10000, 35500),
 (2, 'BMW', 'Germany', 15600, 24500),
 (3, 'Mercedes', 'Germany', 4000, 55500)]

In [11]:
insertion2 = cars.insert().values([
    {'car_name': 'Lada', 'car_country': 'Russia', 'car_mileage': 5000, 'car_price': 20000}
])
print(insertion2)
conn.execute(insertion2)
conn.execute(db.select(cars)).fetchall()

INSERT INTO "Car" (car_name, car_country, car_mileage, car_price) VALUES (:car_name_m0, :car_country_m0, :car_mileage_m0, :car_price_m0)


[(1, 'Audi', 'Germany', 10000, 35500),
 (2, 'BMW', 'Germany', 15600, 24500),
 (3, 'Mercedes', 'Germany', 4000, 55500),
 (4, 'Lada', 'Russia', 5000, 20000)]

In [12]:
select_where = db.select(cars).where(cars.columns.car_mileage < 12000)
print(select_where)
conn.execute(select_where).fetchall()

SELECT "Car".car_id, "Car".car_name, "Car".car_country, "Car".car_mileage, "Car".car_price 
FROM "Car" 
WHERE "Car".car_mileage < :car_mileage_1


[(1, 'Audi', 'Germany', 10000, 35500),
 (3, 'Mercedes', 'Germany', 4000, 55500),
 (4, 'Lada', 'Russia', 5000, 20000)]

In [13]:
insertion3 = cars.insert().values([
    {'car_name': 'Lada', 'car_country': 'Russia', 'car_mileage': 7000, 'car_price': 15000},
    {'car_name': 'Lada', 'car_country': 'Russia', 'car_mileage': 8000, 'car_price': 17000},
    {'car_name': 'Mercedes', 'car_country': 'Germany', 'car_mileage': 8000, 'car_price': 75500}
])
print(insertion3)
conn.execute(insertion3)
conn.execute(db.select(cars)).fetchall()

INSERT INTO "Car" (car_name, car_country, car_mileage, car_price) VALUES (:car_name_m0, :car_country_m0, :car_mileage_m0, :car_price_m0), (:car_name_m1, :car_country_m1, :car_mileage_m1, :car_price_m1), (:car_name_m2, :car_country_m2, :car_mileage_m2, :car_price_m2)


[(1, 'Audi', 'Germany', 10000, 35500),
 (2, 'BMW', 'Germany', 15600, 24500),
 (3, 'Mercedes', 'Germany', 4000, 55500),
 (4, 'Lada', 'Russia', 5000, 20000),
 (5, 'Lada', 'Russia', 7000, 15000),
 (6, 'Lada', 'Russia', 8000, 17000),
 (7, 'Mercedes', 'Germany', 8000, 75500)]

In [14]:
from sqlalchemy import func, desc, delete

In [15]:
select_where = db.select(cars.columns.car_name, func.sum(cars.columns.car_price)).group_by(cars.columns.car_name)
print(select_where)
conn.execute(select_where).fetchall()

SELECT "Car".car_name, sum("Car".car_price) AS sum_1 
FROM "Car" GROUP BY "Car".car_name


[('Audi', 35500), ('BMW', 24500), ('Lada', 52000), ('Mercedes', 131000)]

In [16]:
select_order = db.select(cars.columns.car_name, func.sum(cars.columns.car_price).label('sum_price')).group_by(cars.columns.car_name).order_by(desc(cars.columns.car_name))
print(select_order)
conn.execute(select_order).fetchall()

SELECT "Car".car_name, sum("Car".car_price) AS sum_price 
FROM "Car" GROUP BY "Car".car_name ORDER BY "Car".car_name DESC


[('Mercedes', 131000), ('Lada', 52000), ('BMW', 24500), ('Audi', 35500)]

In [17]:
insertion_c = countrys.insert().values([
    {'country_name': 'Germany', 'country_capital': 'Berlin'},
    {'country_name': 'Russia', 'country_capital': 'Moscow'}
])
print(insertion_c)

INSERT INTO "Country" (country_name, country_capital) VALUES (:country_name_m0, :country_capital_m0), (:country_name_m1, :country_capital_m1)


In [19]:
for i in range(3):
    conn.execute(insertion_c)
conn.execute(db.select(countrys)).fetchall()

[(1, 'Germany', 'Berlin'),
 (2, 'Russia', 'Moscow'),
 (3, 'Germany', 'Berlin'),
 (4, 'Russia', 'Moscow'),
 (5, 'Germany', 'Berlin'),
 (6, 'Russia', 'Moscow'),
 (7, 'Germany', 'Berlin'),
 (8, 'Russia', 'Moscow')]

In [20]:
dele = db.delete(countrys).where(countrys.columns.country_id > 2)
print(dele)
conn.execute(dele)
conn.execute(db.select(countrys)).fetchall()

DELETE FROM "Country" WHERE "Country".country_id > :country_id_1


[(1, 'Germany', 'Berlin'), (2, 'Russia', 'Moscow')]

In [22]:
join_1 = db.select(cars.columns.car_name, countrys.columns.country_capital).join_from(countrys, cars, cars.columns.car_country == countrys.columns.country_name)
print(join_1)
conn.execute(join_1).fetchall()

SELECT "Car".car_name, "Country".country_capital 
FROM "Country" JOIN "Car" ON "Car".car_country = "Country".country_name


[('Audi', 'Berlin'),
 ('BMW', 'Berlin'),
 ('Mercedes', 'Berlin'),
 ('Mercedes', 'Berlin'),
 ('Lada', 'Moscow'),
 ('Lada', 'Moscow'),
 ('Lada', 'Moscow')]

In [23]:
subq = (db.select(cars).where(cars.columns.car_mileage < 12000).cte())
stmt = db.select(subq.columns.car_name, countrys.columns.country_capital).join_from(countrys, subq, subq.columns.car_country == countrys.columns.country_name)
print(stmt)
conn.execute(stmt).fetchall()

WITH anon_1 AS 
(SELECT "Car".car_id AS car_id, "Car".car_name AS car_name, "Car".car_country AS car_country, "Car".car_mileage AS car_mileage, "Car".car_price AS car_price 
FROM "Car" 
WHERE "Car".car_mileage < :car_mileage_1)
 SELECT anon_1.car_name, "Country".country_capital 
FROM "Country" JOIN anon_1 ON anon_1.car_country = "Country".country_name


[('Audi', 'Berlin'),
 ('Mercedes', 'Berlin'),
 ('Lada', 'Moscow'),
 ('Lada', 'Moscow'),
 ('Lada', 'Moscow'),
 ('Mercedes', 'Berlin')]